In [1]:
import numpy as np
import tensorflow as tf

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


TODO: Re-implement [this](https://github.com/tensorflow/models/blob/master/samples/core/get_started/custom_estimator.py)! 

## TensorFlow Architecture

TensorFlow Core:
1. Building the computational graph (a tf.Graph)
2. Running the computational graph (using a tf.Session)

Graphs are composed of:
1. Operations (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
2. Tensors: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors. 

### Tensors

**Important: tf.Tensors do not have values, they are just handles to elements in the computation graph.**

A tensor's rank is its number of dimensions, while its shape is a tuple of integers specifying the array's length along each dimension.  TensorFlow uses numpy arrays to represent tensor values.

In [2]:
test1 = tf.constant([1., 2., 3.])
test2 = tf.constant([[1., 2., 3.], [4., 5., 6.]])
test3 = tf.constant([[[1., 2., 3.]], [[7., 8., 9.]]])

In [3]:
def get_rank_shape(x):
    return len(x.shape), x.shape.as_list()

print(get_rank_shape(test1))
print(get_rank_shape(test2))
print(get_rank_shape(test3))

(1, [3])
(2, [2, 3])
(3, [2, 1, 3])


Once the computation graph has been built, you can run the computation that produces a particular tf.Tensor and fetch the value assigned to it. This is often useful for debugging as well as being required for much of TensorFlow to work.

In [4]:
sess = tf.Session()
with sess.as_default():
    constant = tf.constant([1, 2, 3])
    tensor = constant * constant
    print(tensor.eval())

[1 4 9]


Reshaping tensors...

In [5]:
rank_three_tensor = tf.ones([3, 4, 5])
matrix = tf.reshape(rank_three_tensor, [6, 10])
matrix.eval(session=sess)

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)

### Sessions

**If a tf.Graph is like a .py file, a tf.Session is like the python executable.**

Notice that printing the tensors does not output the values 3.0, 4.0, and 7.0 as you might expect. The above statements only build the computation graph. These tf.Tensor objects just represent the results of the operations that will be run.

In [6]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

sess.run(total)

Tensor("Const_4:0", shape=(), dtype=float32)
Tensor("Const_5:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


7.0

During a call to tf.Session.run any tf.Tensor only has a single value. 

In [7]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.7046287  0.7383052  0.64732885]
[0.77279234 0.33991587 0.05429554]
(array([1.4811295, 1.2722859, 1.157337 ], dtype=float32), array([2.4811296, 2.272286 , 2.157337 ], dtype=float32))


### Feeding

A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later, like a function argument.

In [8]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


### Datasets

To get a runnable tf.Tensor from a Dataset you must first convert it to a tf.data.Iterator, and then call the Iterator's get_next method.  The simplest way to create an Iterator is with the make_one_shot_iterator method. 

In [9]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

[0 1]
[2 3]
[4 5]
[6 7]


Some TensorFlow functions return tf.Operations instead of tf.Tensors. The result of calling run on an Operation is None. You run an operation to cause a side-effect, not to retrieve a value. Examples of this include the initialization, and training ops demonstrated later.

In [10]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[ 0.3845165  0.4968612 -0.5291756]
[-0.9508269  1.6286633 -0.5802473]
[ 0.19768722 -1.2751924  -0.7901949 ]
[-0.35618597 -0.66406137 -2.3090224 ]
[1.7756457 0.5720449 1.1697505]
[-1.3467379  -0.46261194 -1.0825694 ]
[ 1.1338474 -0.7046132  0.8558197]
[-0.40513965  0.24549866  1.2865839 ]
[0.55418175 1.2342918  1.8840885 ]
[-1.384604    0.804429    0.60902476]


### Layers

A trainable model must modify the values in the graph to get new outputs with the same input. Layers are the preferred way to add trainable parameters to a graph.

Layers package together both the variables and the operations that act on them. For example a dense (or fully-connected) layer performs a weighted sum across all inputs for each output and applies an optional activation function. The connection weights and biases are managed by the layer object.

The following code creates a Dense layer that takes a batch of input vectors, and produces a single output value for each. To apply a layer to an input, call the layer as if it were a function.  The layer inspects its input to determine sizes for its internal variables. So here we must set the shape of the x placeholder so that the layer can build a weight matrix of the correct size.

In [11]:
x = tf.placeholder(tf.float32)#, shape=[None, 3])
linear_model = tf.layers.Dense(units=1, kernel_initializer=tf.zeros_initializer, bias_initializer=tf.constant_initializer(1))
y = linear_model(x)

ValueError: Input 0 of layer dense_1 is incompatible with the layer: its rank is undefined, but the layer requires a defined rank.

In [12]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1, kernel_initializer=tf.constant_initializer([1, 2, 3]), 
                               bias_initializer=tf.constant_initializer(1))
y = linear_model(x)

The layer contains variables that must be initialized before they can be used. 

A TensorFlow variable is the best way to represent shared, persistent state manipulated by your program.  Variables are manipulated via the tf.Variable class. A tf.Variable represents a tensor whose value can be changed by running ops on it. Unlike tf.Tensor objects, a tf.Variable exists outside the context of a single session.run call.

**Important: Calling tf.global_variables_initializer only creates and returns a handle to a TensorFlow operation. That op will initialize all the global variables when we run it with tf.Session.run.**

**Also note that this global_variables_initializer only initializes variables that existed in the graph when the initializer was created. So the initializer should be one of the last things added during graph construction.**

In [13]:
init = tf.global_variables_initializer()
sess.run(init)

In [14]:
sess.run(linear_model.weights)

[array([[1.],
        [2.],
        [3.]], dtype=float32), array([1.], dtype=float32)]

In [15]:
sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]})

array([[15.],
       [33.]], dtype=float32)

### Feature Columns

The easiest way to experiment with feature columns is using the tf.feature_column.input_layer function. This function only accepts dense columns as inputs, so to view the result of a categorical column you must wrap it in an tf.feature_column.indicator_column.

Definition of dense column??

In [16]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']
}

department_column = tf.feature_column.categorical_column_with_vocabulary_list('department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

Feature columns can have internal state, like layers, so they often need to be initialized. Categorical columns use lookup tables internally and these require a separate initialization op, tf.tables_initializer.

In [17]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))
sess.run(inputs)

array([[ 1.,  0.,  5.],
       [ 1.,  0., 10.],
       [ 0.,  1.,  8.],
       [ 0.,  1.,  9.]], dtype=float32)

### Putting It All Together

A simple linear regression!

In [18]:
x = tf.placeholder(tf.float32, shape=[None, 1])
y_true = tf.placeholder(tf.float32, shape=[None, 1])

data = {
    x: [[1], [2], [3], [4]],
    y_true: [[0], [-1], [-2], [-3]]
}

linear_model = tf.layers.Dense(units=1,
                               kernel_initializer=tf.zeros_initializer, 
                               bias_initializer=tf.zeros_initializer)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
  _, loss_value = sess.run((train, loss), feed_dict=data)
  print(loss_value)

print(sess.run(y_pred, feed_dict=data))
print(sess.run(linear_model.weights))

3.5
1.69
0.87035024
0.4957618
0.3213858
0.23726682
0.19401541
0.16943617
0.15354478
0.14184044
0.13228557
0.123949364
0.11639896
0.10942595
0.102923475
0.0968312
0.0911102
0.08573199
0.08067339
0.07591425
0.0714363
0.06722271
0.063257694
0.059526626
0.056015614
0.052711725
0.04960269
0.046677034
0.04392396
0.041333247
0.03889534
0.036601223
0.03444241
0.032410946
0.030499289
0.028700395
0.027007595
0.025414642
0.02391563
0.022505064
0.021177668
0.019928578
0.018753154
0.017647067
0.016606199
0.015626743
0.014705041
0.013837724
0.013021546
0.012253509
0.011530784
0.0108506745
0.010210691
0.009608445
0.0090417275
0.008508431
0.008006589
0.0075343386
0.0070899576
0.006671782
0.006278269
0.005907961
0.0055595017
0.0052315886
0.004923025
0.0046326537
0.0043594074
0.0041022855
0.0038603302
0.0036326402
0.0034183827
0.0032167563
0.0030270237
0.002848486
0.0026804795
0.0025223778
0.002373601
0.0022336063
0.0021018598
0.0019778912
0.0018612323
0.0017514552
0.0016481485
0.0015509406
0.0014594619